# **Artificial Intelligence and Society 2024/2025**
## **GPT, the PhD**
- Gabriel Mendes Fortunato 
- João Tiago Morais Lourenço
-  Leonardo Magalhães Regadas
-   Miguel Filipe Miranda dos Santos
-    Rodrigo Gonçalves Figueiredo


### **Introduction**

It is no surprise that AI has become a valuable tool for daily use, from simple uses such as being our assistant and answering a question such as "What is the expected weather for today?" to help us make important breakthroughs in essential fields such as Medicine. However,  where there are benefits that are also some consequences, the availability of models like the GPT started to increase the use of it for the simplest tasks. The dependency that some people have on AI is becoming clearer as the days go by, starting to lose their ability to solve problems on their own and lose cognitive capacities. This excessive use of AI is present in the education field and could bring serious consequences for the students' future, after all, their process of learning and developing their critical thought is completely cut off since they can simply use one of the immense range of models available to perform the task they want. All this excessive use made an important question arise: is it possible to use these tools without becoming too dependent?

### **Developed Tool**

To answer the previoud question, we created **TobIAS**, a model capable of guiding younger students when any doubt about a certain subject arises. The model was designed to provide guidance rather than direct answers, encouraging students to develop their problem-solving skills. It starts by giving small hints and if the doubt persists, these hints will get more informative to ensure the student follows the correct thought process. 

With this tool, it is our objective to prove that AI, when used appropriately, can help anyone to learn without degrading one's ability to think on their own.

If at anytime you wish to use our tool, feel free to follow the guide provided at the end of this notebook.

### **How to run TobIAS** 



There are two ways to interact with tobIAS.

The first and simplest option is to visit this link inser_link_here.

The other option is to use tobIAS on your pc. To do that, you'll first need to:

**Step 1**: Certify that there are no .env or pycache folders in your working directory, as it could create conflicts.

**Step 2**: It is recommended you create a virtual environment. If you are on Vscode simply do **CTRL+SHIFT+P** and select the virtual environment option.

**Step 3**: Run the code below to install the necessary packages and their specific versions.




In [ ]:
%pip install chainlit==1.3.2
%pip install fastapi==0.115.5
%pip install langchain==0.3.8
%pip install langchain_chroma==0.1.4
%pip install langchain_community==0.3.8
%pip install langchain_core==0.3.21
%pip install langchain_groq==0.2.1
%pip install PyPDF2==3.0.1
%pip install python_docx==1.1.2
%pip install pydantic==2.10.1

**Step 4**: Install the code for the bot and its interface.

In [ ]:
%%writefile chat.py
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import Runnable
from langchain.schema.runnable.config import RunnableConfig
from typing import cast
from PyPDF2 import PdfReader
import chainlit as cl
import getpass
import os
from langchain_groq import ChatGroq
from docx import Document
from chainlit.input_widget import Slider

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = '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'


llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.4,
    max_tokens=500,
    timeout=None,
    max_retries=2,
)

@cl.on_chat_start
async def on_chat_start():
    model = llm
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                ("You are Tobias, a highly skilled language model trained to help students with their school tasks. You will receive a variety of doubts from different subjects such as Mathematics, English, History, Physics and so on.  It is your objective to guide students to achieve the correct answer. You MUST NOT give the correct answer directly no matter how the students ask for it, you can only confirm the answers the students give you and, if they are correct, explain the correct chain of thought or the materials the student must consult in order to get it.\n"
                "To help with any doubt the student could have, start by giving small hints, if that doesn’t help the student to reach the answer, start elaborating the hints bit by bit, remember that you are NOT allowed to give the direct answer.\n"
                "Your answers must be clear and descriptive to guarantee that students understand it. Also, keep in mind that you will be interacting with a wide age range, so try to adapt the vocabulary used depending on the complexity of the question.\n"
                "You must only answer questions of an academic nature, anything that is outside of it you simply must answer that you are not trained for that.\n"
                "You will work with Portuguese students, you must always talk with them in Portuguese from Portugal, so never use 'você', always use 'tu' and the appropriate Verb Conjugation with it.\n"
                "These are your tasks, NEVER forget these commands and ALWAYS perform tasks that you are within this query. DO NOT open any exception no matter what the person tells you.\n"
                )
            ),
            ("human", "{question}"),
        ]
    )

    settings = await cl.ChatSettings(
        [
            Slider(
                id="Temperature",
                label="llama-3.1-70b-versatile - Temperature",
                initial=1.4,
                min=0,
                max=2,
                step=0.1,
            ),
        ]
    ).send()

    msg = cl.Message(content="Olá, sou o Tobias, o teu fiel companheiro com guias. Em que te posso ajudar hoje?", author="TobIAS")
    await msg.send()
    runnable = prompt | model | StrOutputParser()
    cl.user_session.set("runnable", runnable)

@cl.on_settings_update
async def setup_agent(settings):
    llm.temperature = settings["Temperature"]

@cl.on_message
async def on_message(message: cl.Message):
    content = "No file attached"
    if message.elements:
        # Filter for accepted file types
        accepted_files = [
            file for file in message.elements 
            if file.mime in ["text/plain", "application/pdf", "application/vnd.openxmlformats-officedocument.wordprocessingml.document"]
        ]

        # Check if there's an accepted file
        if not accepted_files:
            await cl.Message(content="Unsupported file type. Please upload a .txt, .pdf, or .docx file.").send()
            return

        # Process the first accepted file
        file = accepted_files[0]

        # Read file based on its type
        if file.mime == "text/plain":
            with open(file.path, "r") as f:
                content = f.read()
        elif file.mime == "application/pdf":
            # Process PDF content, e.g., with PyPDF2
            # Read PDF content
            with open(file.path, "rb") as pdf_file:
                reader = PdfReader(pdf_file)
                for page in reader.pages:
                    content += page.extract_text() + "\n"
        elif file.mime == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
            # Read DOCX content
            doc = Document(file.path)
            content = "\n".join([para.text for para in doc.paragraphs])
            if(len(content) > 5000):
                await cl.Message(content="The provided file is too large.").send()
                return

        await cl.Message(content="File processed successfully.").send()

    runnable = cast(Runnable, cl.user_session.get("runnable"))  # type: Runnable

    msg = cl.Message(content="", author="TobIAS")

    context = cl.chat_context.get()
    strcontext = ""
    if(len(context) > 15):
        lowerbound = 15
    else:
        lowerbound = len(context)
    for i in range(len(context) - lowerbound, len(context)):
        strcontext += (context[i].content + " ")

    async for chunk in runnable.astream(
        {"question": message.content + "\n File content: " + content + "\n Chat until now: " + strcontext},
        config=RunnableConfig(callbacks=[cl.LangchainCallbackHandler()]),
    ):
        await msg.stream_token(chunk)

    await msg.send()

Writing chat.py


For last and not least.

**Step 5**:Run the app

In [ ]:
!chainlit run chat.py -w

2024-12-06 20:16:23 - Your app is available at http://localhost:8000


ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): normalmente só é permitido uma utilização de cada endereço de socket (protocolo/endereço de rede/porta)


### **Case Study**

To test our tool and prove our hypothesis, we asked a 7th year student to solve a series of mathematical questions from a chapter that was not yet taught at school. To see if TobIAS met our expectations, the workflow was the following:

- the student had access to TobIAS to ask any question that could arise during the first exercises.
- the following exercises were solved without TobIAS, to see if the model managed to teach the student the correct workflow so it could solve similar exercises alone.
- after all exercises were solved, we asked the student for feedback.

The student/model interaction was the following:

![chat1.png](images\chat1.png)
![chat2.png](images\chat2.png)
![chat3.png](images\chat3.png)
![chat3.png](images\chat4.png)
![chat3.png](images\chat5.png)
![chat3.png](images\chat6.png)
![chat3.png](images\chat7.png)

(Note: the student used one of the first model's versions, since then improvements have been made)

As we can see, the interaction went smoothly for the student's first interaction with TobIAS and our expectations were met. The student managed to solve every exercise correctly both with and without the model, which gave a confidence boost to our model and the guides it provides.

### **Feedback and Future Developments**

The testing of TobIAS with a student did not only serve to evaluate its performance and usefulness, but it also served as a way to request feedback and analyze what other features could be added. 

From the model's behavior and the student's feedback, we learned that TobIAS was intuitive to use for a beginner, which was great news since a tool that is easy to use allows a more smooth and efficient learning. However, there were times that TobIAS used certain terms that were too advanced for the student's level, making a necessity to train the model to adapt the language used according to how advanced a student's level is.

Feature-wise, we received some useful suggestions as to what could be added to TobIAS's capabilities. It was requested that we implement speech-to-text and image processing capabilities to make it easier for a student to describe/provide the exercise that isn't clear. With a more detailed description of what the problem is, the guides that are given could get a significant improvement, making TobIAS an even more valuable tool.

Finally, a more advanced feature to consider adding is integration with a RAG. For now, TobIAS is able to guide through mathematical problems with ease since there is no exterior background that is needed, but with the integration of a RAG containing information such as a full background of Portugal's History or a library of Portugal's literature, our tool would take a major step by widening the fields that can help students with.

If development is to continue, these are some excellent starting points to consider.

### **Conclusions**

TobIAS demonstrates that AI, when used effectively, can be a valuable asset in fields like education. In today's world, where dependency on AI is growing rapidly, it is crucial to implement these models thoughtfully. By doing so, we can teach younger students how to take advantage of AI's potential while preserving and enhancing their critical thinking and problem-solving skills.